# Enhanced ICP: soft correspondences with Generalized-ICP

Standard ICP and point-to-plane ICP treat all correspondences as hard constraints with equal weight. This makes them sensitive to incorrect matches--a single bad correspondence gets the same influence as a good one.

**Generalized-ICP (GICP)** addresses this through *soft correspondences*: matches are weighted by their geometric consistency. The algorithm uses local surface structure from both scans to automatically down-weight unreliable correspondences.

In [1]:
import numpy as np
from scipy.spatial import KDTree

from pydrake.all import (
    PointCloud,
    Rgba,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    StartMeshcat,
)

from manipulation import FindResource
from manipulation.icp import IterativeClosestPoint

import matplotlib.pyplot as plt

/usr/local/lib/python3.12/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


## Stanford bunny point clouds

We'll use the Stanford Bunny, transformed with a known rotation and translation plus noise.

In [5]:
model_pcl = np.load(FindResource("models/bunny/bunny.npy"))

X_WO_true = RigidTransform(
    RotationMatrix.MakeXRotation(np.pi / 6),
    [-0.02, 0.02, 0.02]
)

scene_pcl = X_WO_true.multiply(model_pcl)
np.random.seed(42)
scene_pcl += np.random.randn(*scene_pcl.shape) * 0.0005

print(f"Model shape: {model_pcl.shape}")
print(f"Scene shape: {scene_pcl.shape}")

cloud = PointCloud(model_pcl.shape[1])
cloud.mutable_xyzs()[:] = model_pcl

meshcat.Delete()
meshcat.SetProperty("/Background", "visible", False)
meshcat.SetProperty("/Cameras/default/rotated/<object>", "zoom", 10.5)
meshcat.SetObject("model", cloud, point_size=0.01, rgba=Rgba(0, 0, 1))
meshcat.SetTransform("model", RigidTransform())

cloud2 = PointCloud(scene_pcl.shape[1])
cloud2.mutable_xyzs()[:] = scene_pcl
meshcat.SetObject("scene", cloud2, point_size=0.01, rgba=Rgba(1, 0, 0))
meshcat.SetTransform("scene", RigidTransform())

Model shape: (3, 8171)
Scene shape: (3, 8171)


## The probabilistic framework

GICP models each point as drawn from a Gaussian: $p_i \sim N(\hat{p}_i, C_i)$

The covariance $C_i$ encodes local surface structure:
- Small variance along the surface normal (we know position accurately)
- Large variance in the tangent plane (uncertain about exact position on surface)

$$C_i = R \begin{bmatrix} \epsilon & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{bmatrix} R^T$$

where $R$ aligns with the surface normal and $\epsilon = 0.001$.

GICP then minimizes the Mahalanobis distance:

$$T^* = \arg\min_T \sum_i d_i^T (C_i^A + T C_i^B T^T)^{-1} d_i$$

When normals are aligned, the combined covariance stays thin (strong constraint). When normals differ, it becomes isotropic (weak constraint). This automatic reweighting is the key to robustness.

## Computing surface normals

In [ ]:
def compute_normals(points, k_neighbors=20):
    """Estimate surface normals using local neighborhoods."""
    kdtree = KDTree(points.T)
    normals = np.zeros_like(points)
    k = min(k_neighbors, points.shape[1])
    
    for i in range(points.shape[1]):
        # Find k nearest neighbors for each point
        _, indices = kdtree.query(points[:, i], k=k)
        if np.isscalar(indices):
            indices = [indices]
        
        # Fit a plane through the neighbors - normal is the eigenvector 
        # corresponding to the smallest eigenvalue
        neighbors = points[:, indices]
        centered = neighbors - np.mean(neighbors, axis=1, keepdims=True)
        cov = centered @ centered.T
        
        eigvalues, eigvecs = np.linalg.eigh(cov)
        normals[:, i] = eigvecs[:, 0] / np.linalg.norm(eigvecs[:, 0])
    
    return normals


print("Computing surface normals...")
model_normals = compute_normals(model_pcl)
scene_normals = compute_normals(scene_pcl)
print(f"Done: {model_normals.shape[1]} model normals, {scene_normals.shape[1]} scene normals")

## Computing covariance matrices

Each point's covariance matrix encodes the local surface geometry. We construct it by:

1. **Building a local coordinate frame**: Given a surface normal $\mathbf{n}_i$, we construct an orthonormal basis $[\mathbf{n}_i, \mathbf{t}_1, \mathbf{t}_2]$ where $\mathbf{t}_1$ and $\mathbf{t}_2$ are tangent vectors in the surface plane.

2. **Defining anisotropic uncertainty**: In this local frame, we assign small variance $\varepsilon$ along the normal direction (high confidence) and unit variance in the tangent directions (low confidence):

$$\Lambda = \begin{bmatrix} \varepsilon & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{bmatrix}, \quad \varepsilon \ll 1$$

3. **Transforming to world coordinates**: The covariance matrix in world coordinates is:

$$C_i = R_i \Lambda R_i^T$$

where $R_i = [\mathbf{n}_i \mid \mathbf{t}_1 \mid \mathbf{t}_2]$ is the rotation matrix formed by the orthonormal basis.

This construction ensures that the uncertainty is:
- **Minimal** along the normal direction (we know the point lies *on* the surface)
- **Maximal** in the tangent plane (we're uncertain about the *exact* position along the surface)

The ratio $1/\varepsilon$ determines how strongly the surface orientation constrains the alignment.

In [ ]:
def compute_covariances(points, normals, epsilon=0.001):
    """Build covariance matrices that encode local surface geometry."""
    covariances = []
    
    for i in range(points.shape[1]):
        n = normals[:, i] / np.linalg.norm(normals[:, i])
        
        # Build orthonormal basis [n, t1, t2] aligned with the surface
        # Start with an arbitrary vector that's not parallel to n
        if abs(n[0]) < 0.9:
            v = np.array([1, 0, 0])
        else:
            v = np.array([0, 1, 0])
        
        # Gram-Schmidt to get tangent vectors
        t1 = v - (v @ n) * n
        t1 = t1 / np.linalg.norm(t1)
        t2 = np.cross(n, t1)
        
        # Construct covariance: small variance along normal, large in tangent plane
        R = np.column_stack([n, t1, t2])
        C_local = np.diag([epsilon, 1.0, 1.0])
        C = R @ C_local @ R.T
        
        covariances.append(C)
    
    return covariances

model_covariances = compute_covariances(model_pcl, model_normals)
scene_covariances = compute_covariances(scene_pcl, scene_normals)

# Verify the anisotropy - should see epsilon and two 1.0s
eigvals = np.linalg.eigvalsh(model_covariances[100])
print(f"Example eigenvalues: {eigvals} (small/large ratio = {eigvals[2]/eigvals[0]:.0f})")

## Generalized ICP implementation

GICP solves a weighted least-squares problem at each iteration. Given current estimate $T$ (rotation $R$, translation $\mathbf{t}$), point correspondences, and covariance matrices, we seek an incremental update $\Delta T$ that minimizes:

$$E(T) = \sum_{i=1}^{N} \mathbf{d}_i^T (C_i^M + C_{c_i}^S)^{-1} \mathbf{d}_i$$

where:
- $\mathbf{d}_i = R\mathbf{p}_i^M + \mathbf{t} - \mathbf{p}_{c_i}^S$ is the residual between transformed model point and its scene correspondence
- $C_i^M$ is the covariance of model point $i$ (transformed to world frame: $R C_i^M R^T$)
- $C_{c_i}^S$ is the covariance of the corresponding scene point
- $(C_i^M + C_{c_i}^S)^{-1}$ is the information matrix (inverse covariance) that weights each correspondence

### Linearization

We parameterize the incremental transformation using:
- **Rotation update**: axis-angle representation $\boldsymbol{\omega} \in \mathbb{R}^3$ 
- **Translation update**: $\boldsymbol{\Delta t} \in \mathbb{R}^3$

For small updates, the transformed point linearizes as:

$$T(\mathbf{p}) \approx \mathbf{p} + [\boldsymbol{\omega}]_\times \mathbf{p} + \boldsymbol{\Delta t}$$

where $[\boldsymbol{\omega}]_\times$ is the skew-symmetric matrix:

$$[\boldsymbol{\omega}]_\times = \begin{bmatrix} 0 & -\omega_z & \omega_y \\ \omega_z & 0 & -\omega_x \\ -\omega_y & \omega_x & 0 \end{bmatrix}$$

### Jacobian

The Jacobian of the residual with respect to the 6-DOF update parameters is:

$$J_i = \frac{\partial \mathbf{d}_i}{\partial [\boldsymbol{\omega}^T, \boldsymbol{\Delta t}^T]^T} = \begin{bmatrix} -[\mathbf{p}_i]_\times & I_{3\times3} \end{bmatrix} \in \mathbb{R}^{3 \times 6}$$

where $[\mathbf{p}_i]_\times$ captures how rotation around each axis moves the point, and $I$ is the identity (translation directly adds to position).

### Weighted least-squares solution

We convert the problem to standard least-squares form by applying the Cholesky decomposition $C^{-1} = L L^T$:

$$\min_{\mathbf{x}} \sum_i \| L_i (J_i \mathbf{x} + \mathbf{d}_i) \|^2$$

Solving the normal equations $J^T W J \mathbf{x} = -J^T W \mathbf{d}$ gives us the optimal update $\mathbf{x} = [\boldsymbol{\omega}^T, \boldsymbol{\Delta t}^T]^T$, which we convert back to a rigid transformation and compose with the current estimate.

In [ ]:
def generalized_icp(model_pts, scene_pts, model_cov, scene_cov, 
                   initial_transform, max_iterations=30, tolerance=1e-6):
    """Run GICP with soft correspondences weighted by covariance."""
    T = initial_transform
    errors = []
    
    for iteration in range(max_iterations):
        # Transform model points with current estimate
        model_transformed = T.multiply(model_pts)
        
        # Find nearest neighbors in the scene
        kdtree_scene = KDTree(scene_pts.T)
        dists, idxs = kdtree_scene.query(model_transformed.T)
        correspondences = scene_pts[:, idxs]
        
        # Build weighted least-squares system
        A = []
        b = []
        total_error = 0
        
        R_mat = T.rotation().matrix()
        
        for i in range(model_transformed.shape[1]):
            p_model_tf = model_transformed[:, i]
            p_scene_corresp = correspondences[:, i]
            
            # Combined covariance (transform model cov to world frame first)
            C_model_world = R_mat @ model_cov[i] @ R_mat.T
            C_combined = C_model_world + scene_cov[idxs[i]] + np.eye(3) * 1e-6
            C_inv = np.linalg.inv(C_combined)
            
            # Residual and Mahalanobis distance
            r = p_model_tf - p_scene_corresp
            total_error += r.T @ C_inv @ r
            
            # Jacobian: derivatives w.r.t. [omega; delta_t]
            # Rotation part uses skew-symmetric matrix of the point
            p_cross = np.array([
                [0, -p_model_tf[2], p_model_tf[1]],
                [p_model_tf[2], 0, -p_model_tf[0]],
                [-p_model_tf[1], p_model_tf[0], 0]
            ])
            
            J = np.hstack([-p_cross, np.eye(3)])
            
            # Weight by sqrt of information matrix (Cholesky)
            try:
                L = np.linalg.cholesky(C_inv)
            except:
                # Fallback if not positive definite
                eigvals, eigvecs = np.linalg.eigh(C_inv)
                L = eigvecs @ np.diag(np.sqrt(np.maximum(eigvals, 1e-10))) @ eigvecs.T
            
            A.append(L @ J)
            b.append(-L @ r)
        
        # Solve normal equations: (A^T A) x = A^T b
        A_mat = np.vstack(A)
        b_vec = np.hstack(b)
        
        x = np.linalg.solve(A_mat.T @ A_mat + np.eye(6) * 1e-8, A_mat.T @ b_vec)
        omega = x[:3]  # axis-angle rotation
        t_update = x[3:]  # translation
        
        # Convert axis-angle to rotation matrix
        angle = np.linalg.norm(omega)
        if angle > 1e-10:
            axis = omega / angle
            K = np.array([
                [0, -axis[2], axis[1]],
                [axis[2], 0, -axis[0]],
                [-axis[1], axis[0], 0]
            ])
            R_inc = np.eye(3) + np.sin(angle) * K + (1 - np.cos(angle)) * (K @ K)
            R_inc_drake = RotationMatrix(R_inc)
        else:
            R_inc_drake = RotationMatrix.Identity()
        
        # Compose the update with current transform
        T = RigidTransform(R_inc_drake, t_update).multiply(T)
        
        avg_error = total_error / model_transformed.shape[1]
        errors.append(avg_error)
        
        # Check convergence
        if iteration > 0 and abs(errors[-2] - errors[-1]) < tolerance:
            break
        if np.linalg.norm(omega) < 1e-6 and np.linalg.norm(t_update) < 1e-6:
            break
    
    return T, errors

## Test 1: accuracy with small initialization error

In [ ]:
initial_guess = RigidTransform(
    RotationMatrix.MakeXRotation(np.pi / 6 + 0.15),
    [-0.015, 0.017, 0.024]
)

X_init_error = initial_guess.inverse().multiply(X_WO_true)
print(f"Initial guess error: {np.linalg.norm(X_init_error.translation())*1000:.1f} mm, "
      f"{abs(RollPitchYaw(X_init_error.rotation()).vector()[0])*180/np.pi:.1f} deg\n")

T_gicp, _ = generalized_icp(model_pcl, scene_pcl, model_covariances, scene_covariances, initial_guess)
T_standard, _ = IterativeClosestPoint(p_Om=model_pcl, p_Ws=scene_pcl, X_Ohat=initial_guess, max_iterations=25)

gicp_error = np.linalg.norm(T_gicp.inverse().multiply(X_WO_true).translation())
std_error = np.linalg.norm(T_standard.inverse().multiply(X_WO_true).translation())

print(f"\nStandard ICP error: {std_error*1000:.3f} mm")
print(f"GICP error:         {gicp_error*1000:.3f} mm")

## Test 2: robustness to noisy initialization

The key advantage of GICP is robustness. Let's test with large initialization errors.

In [ ]:
def test_robustness(num_trials=10, translation_noise=0.1, rotation_noise=0.15):
    results = {"standard": [], "gicp": []}
    
    print(f"Testing with {num_trials} trials (noise: +/-{translation_noise*1000:.0f} mm, +/-{rotation_noise*180/np.pi:.0f} deg)\n")
    
    for trial in range(num_trials):
        t_noisy = X_WO_true.translation() + np.random.randn(3) * translation_noise
        angle_noisy = np.pi / 6 + np.random.randn() * rotation_noise
        
        noisy_guess = RigidTransform(RotationMatrix.MakeXRotation(angle_noisy), t_noisy)

        T_std, _ = IterativeClosestPoint(p_Om=model_pcl, p_Ws=scene_pcl, X_Ohat=noisy_guess, max_iterations=25)
        error_std = np.linalg.norm(T_std.inverse().multiply(X_WO_true).translation())
        results["standard"].append(error_std)

        T_g, _ = generalized_icp(model_pcl, scene_pcl, model_covariances, scene_covariances, noisy_guess, max_iterations=30)
        error_g = np.linalg.norm(T_g.inverse().multiply(X_WO_true).translation())
        results["gicp"].append(error_g)

        print(f"[Iteration {trial:>2}] ICP error: {error_std*1000:.3f} mm, GICP error: {error_g*1000:.3f} mm")
    
    std_success = [e for e in results["standard"] if e < 0.005]
    gicp_success = [e for e in results["gicp"] if e < 0.005]
    
    print("Results:")
    print(f"Standard ICP: {len(std_success)}/{num_trials} succeeded")
    if std_success:
        print(f"  Mean error: {np.mean(std_success)*1000:.3f} +/- {np.std(std_success)*1000:.3f} mm")
    
    print(f"GICP:         {len(gicp_success)}/{num_trials} succeeded")
    if gicp_success:
        print(f"  Mean error: {np.mean(gicp_success)*1000:.3f} +/- {np.std(gicp_success)*1000:.3f} mm")
    
    return results


results = test_robustness(num_trials=10, translation_noise=0.1)

## Visualizing soft correspondences

Let's examine how GICP weights correspondences based on normal alignment.

In [ ]:
# Transform model covariances and normals to world frame
R_mat = T_gicp.rotation().matrix()
model_transformed = T_gicp.multiply(model_pcl)

# Find correspondences after GICP convergence
kdtree = KDTree(scene_pcl.T)
distances, indices = kdtree.query(model_transformed.T)

weights = []
normal_alignments = []

# Sample some correspondences to analyze
for i in range(min(500, model_pcl.shape[1])):
    C_model_world = R_mat @ model_covariances[i] @ R_mat.T
    C_combined = C_model_world + scene_covariances[indices[i]]
    
    # Weight = 1/smallest eigenvalue (strongest constraint direction)
    eigvals = np.linalg.eigvalsh(C_combined)
    weights.append(1.0 / eigvals[0])
    
    # How aligned are the surface normals?
    n_model_world = (R_mat @ model_normals[:, i])
    n_scene = scene_normals[:, indices[i]]
    normal_alignments.append(abs(n_model_world @ n_scene))

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(normal_alignments, weights, alpha=0.5, s=10) 
plt.xlabel("Normal alignment |n1 * n2|", fontsize=12)
plt.ylabel("Constraint weight (1/lambda_min)", fontsize=12)
plt.title("Soft correspondences", fontsize=13)
plt.grid(True, alpha=0.3)
plt.yscale("log")

plt.subplot(1, 2, 2)
plt.hist(weights, bins=30, alpha=0.7, edgecolor="black")
plt.xlabel("Constraint weight", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.title("Weight distribution", fontsize=13)
plt.xscale("log")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

corr = np.corrcoef(normal_alignments, weights)[0, 1]
print(f"Correlation between normal alignment and weight: {corr:.3f}")
print("(Positive correlation confirms: aligned normals -> stronger constraints)")

**Summary.** While both ICP and GICP work well with good initialization, GICP significantly outperforms standard ICP with noisy initialization by assigning higher weights to normals that are aligned.

### Why GICP works better

The probabilistic framework lets GICP encode surface geometry in covariance matrices. When correspondences are geometrically inconsistent (misaligned normals), they're automatically down-weighted. This makes the algorithm naturally robust to incorrect matches without manual parameter tuning.

This tutorial is based on ["Generalized-ICP" by Aleksandr V. Segal et al. (2009)](https://www.robots.ox.ac.uk/~avsegal/resources/papers/Generalized_ICP.pdf)